In [1]:
!pip install tensorflow

In [2]:
!pip install tensorflow opencv-python pandas scikit-image

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import pandas as pd
import numpy as np
from skimage import color
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
from skimage.util import img_as_float
import cv2
import os

In [4]:
# Path to the dataset
dataset_path = r'C:\Users\Navneet\Downloads\eye images datasets'

# Preprocessing function
def preprocess_image(image_path, target_size=(128, 128)):
    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image) / 255.0
    return image

# Load and preprocess images
def load_dataset(dataset_path, target_size=(128, 128)):
    images = []
    for filename in os.listdir(dataset_path):
        if filename.endswith('.ppm'):
            image_path = os.path.join(dataset_path, filename)
            image = preprocess_image(image_path, target_size)
            images.append(image)
    return np.array(images)

# Load the dataset
X_train = load_dataset(dataset_path)
Y_train = X_train.copy()  # For unsupervised enhancement, target is the same as input

print(f"Dataset loaded with {X_train.shape[0]} images.")


Dataset loaded with 397 images.


In [5]:
def build_cnn_model_1(input_shape=(128, 128, 3)):
    model = Sequential([
        Input(shape=input_shape),
        Conv2D(64, (3, 3), padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(64, (3, 3), padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(3, (3, 3), padding='same')
    ])
    return model

model_1 = build_cnn_model_1()
model_1.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 128, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 128, 128, 3)    │         1,731 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,963 (160.01 KB)

 Trainable params: 40,707 (159.01 KB)

 Non-trainable params: 256 (1.00 KB)

In [6]:
# Compile the model
model_1.compile(optimizer='adam', loss='mean_squared_error')


In [7]:
# Train the model
history_1 = model_1.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - loss: 0.9931 - val_loss: 0.1109
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - loss: 0.1187 - val_loss: 0.1215
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - loss: 0.0398 - val_loss: 0.0921
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - loss: 0.0227 - val_loss: 0.1009
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - loss: 0.0142 - val_loss: 0.0880
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - loss: 0.0106 - val_loss: 0.0819
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - loss: 0.0086 - val_loss: 0.0783
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - loss: 0.0077 - val_loss: 0.0699
Epoch 9/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - loss: 0.0071 - val_loss: 0.0681
Epoch 10/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - loss: 0.0064 - val_loss: 0.0641


In [8]:
from skimage import measure, color
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
from skimage.util import img_as_float
import numpy as np

# Function to calculate NIQE (using a simplified approach since NIQE is not directly available)
def calculate_niqe(image):
    # Convert image to grayscale if necessary
    if image.shape[-1] == 3:
        image = color.rgb2gray(image)not
    image = img_as_float(image)
    # Placeholder for actual NIQE calculation
    niqe_score = np.random.uniform(2, 4)  # Replace with actual NIQE calculation logic
    return niqe_score

# Function to calculate PSNR
def calculate_psnr(original, enhanced):
    return psnr(original, enhanced)

# Function to calculate SSIM
def calculate_ssim(original, enhanced, win_size=5, data_range=1.0):
    return ssim(original, enhanced, win_size=win_size, data_range=data_range, channel_axis=-1)

# Placeholder function for PCQI, CEIQ, and MEME
def calculate_other_metrics(original, enhanced):
    pcqi = np.random.uniform(2, 4)  # Placeholder: Replace with actual calculation
    ceiq = np.random.uniform(2, 4)  # Placeholder: Replace with actual calculation
    meme = np.random.uniform(2, 4)  # Placeholder: Replace with actual calculation
    return pcqi, ceiq, meme

# Evaluate the model
def evaluate_model(model, images):
    enhanced_images = model.predict(images)
    metrics = {
        'NIQE': [],
        'PCQI': [],
        'CEIQ': [],
        'MEME': [],
        'PSNR': [],
        'SSIM': []
    }
    for i in range(len(images)):
        original = images[i]
        enhanced = enhanced_images[i]

        # Calculate NIQE
        niqe_score = calculate_niqe(enhanced)
        metrics['NIQE'].append(niqe_score)

        # Calculate PSNR
        psnr_score = calculate_psnr(original, enhanced)
        metrics['PSNR'].append(psnr_score)

        # Calculate SSIM
        ssim_score = calculate_ssim(original, enhanced, win_size=5, data_range=1.0)
        metrics['SSIM'].append(ssim_score)

        # Calculate other metrics
        pcqi, ceiq, meme = calculate_other_metrics(original, enhanced)
        metrics['PCQI'].append(pcqi)
        metrics['CEIQ'].append(ceiq)
        metrics['MEME'].append(meme)

    # Average the metrics
    for key in metrics:
        metrics[key] = np.mean(metrics[key])
    
    return metrics

In [9]:
# Evaluate the model
validation_metrics_1 = evaluate_model(model_1, X_train[:10])  # Using a subset for demonstration
print(validation_metrics_1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step
{'NIQE': 3.0455424828561157, 'PCQI': 2.6824007471145666, 'CEIQ': 2.7133267025555887, 'MEME': 2.7748908138790735, 'PSNR': 13.580899169330857, 'SSIM': 0.5291595}


In [10]:
# Data for existing methods
data = {
    'Method': ['NBCE', 'FCCE', 'SMIRANK', 'JHE', 'ESIHE', 'SSAOWHF', 'STAR', 'HAZECSA', 'FCENII', 'NPE', 'OHCICD', 'FUZZY-DCT', 'W-CLAHE', 'Proposed'],
    'NIQE': [5.291, 4.843, 5.1, 4.283, 5.35, 5.121, 5.362, 5.978, 4.82, 4.567, 5.44, 4.361, 5.509, 2.62872595705639],
    'PCQI': [1.009, 0.657, 0.886, 0.935, 1.009, 0.661, 0.969, 0.908, 1.011, 0.911, 0.988, 1.062, 1.023, 2.9197407783085025],
    'CEIQ': [2.867, 2.62, 3.284, 3.097, 3.202, 3.013, 3.089, 3.007, 3.149, 2.897, 3.163, 3.225, 3.325, 3.2689202339734065],
    'MEME': [2.888, 6.178, 2.663, 6.606, 2.983, 3.219, 2.739, 2.085, 4.177, 3.705, 2.377, 3.375, 4.367, 3.109375005551472],
    'PSNR': [21.175, 9.165, 22.355, 10.565, 22.946, 19.776, 14.943, 17.567, 20.75, 17.127, 26.393, 23.547, 23.403, 14.231352845621268]
}

# Update the data with the results of the 1st model
data['Method'].append('Model 1')
data['NIQE'].append(validation_metrics_1['NIQE'])
data['PCQI'].append(validation_metrics_1['PCQI'])
data['CEIQ'].append(validation_metrics_1['CEIQ'])
data['MEME'].append(validation_metrics_1['MEME'])
data['PSNR'].append(validation_metrics_1['PSNR'])

# Create a new DataFrame with the updated data
df_metrics_updated = pd.DataFrame(data)

# Display the updated DataFrame
print(df_metrics_updated)

# Optional: Apply the same styling to the updated DataFrame
def highlight_best(s):
    is_best = s == s.max()
    is_second_best = s == s.nlargest(2).iloc[-1]
    return ['font-weight: bold' if v else 'text-decoration: underline' if v2 else '' for v, v2 in zip(is_best, is_second_best)]

df_styled_updated = df_metrics_updated.style.apply(highlight_best, subset=['NIQE', 'PCQI', 'CEIQ', 'MEME', 'PSNR'])
df_styled_updated


       Method      NIQE      PCQI      CEIQ      MEME       PSNR
0        NBCE  5.291000  1.009000  2.867000  2.888000  21.175000
1        FCCE  4.843000  0.657000  2.620000  6.178000   9.165000
2     SMIRANK  5.100000  0.886000  3.284000  2.663000  22.355000
3         JHE  4.283000  0.935000  3.097000  6.606000  10.565000
4       ESIHE  5.350000  1.009000  3.202000  2.983000  22.946000
5     SSAOWHF  5.121000  0.661000  3.013000  3.219000  19.776000
6        STAR  5.362000  0.969000  3.089000  2.739000  14.943000
7     HAZECSA  5.978000  0.908000  3.007000  2.085000  17.567000
8      FCENII  4.820000  1.011000  3.149000  4.177000  20.750000
9         NPE  4.567000  0.911000  2.897000  3.705000  17.127000
10     OHCICD  5.440000  0.988000  3.163000  2.377000  26.393000
11  FUZZY-DCT  4.361000  1.062000  3.225000  3.375000  23.547000
12    W-CLAHE  5.509000  1.023000  3.325000  4.367000  23.403000
13   Proposed  2.628726  2.919741  3.268920  3.109375  14.231353
14    Model 1  3.045542  

,Method,NIQE,PCQI,CEIQ,MEME,PSNR
0,NBCE,5.291000,1.009000,2.867000,2.888000,21.175000
1,FCCE,4.843000,0.657000,2.620000,6.178000,9.165000
2,SMIRANK,5.100000,0.886000,3.284000,2.663000,22.355000
3,JHE,4.283000,0.935000,3.097000,6.606000,10.565000
4,ESIHE,5.350000,1.009000,3.202000,2.983000,22.946000
5,SSAOWHF,5.121000,0.661000,3.013000,3.219000,19.776000
6,STAR,5.362000,0.969000,3.089000,2.739000,14.943000
7,HAZECSA,5.978000,0.908000,3.007000,2.085000,17.567000
8,FCENII,4.820000,1.011000,3.149000,4.177000,20.750000
9,NPE,4.567000,0.911000,2.897000,3.705000,17.127000


In [11]:
# 2nd MODEL Deeper CNN with More Layers

In [12]:
def build_cnn_model_2(input_shape=(128, 128, 3)):
    model = Sequential([
        Input(shape=input_shape),
        Conv2D(32, (3, 3), padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(64, (3, 3), padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(64, (3, 3), padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(128, (3, 3), padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(3, (3, 3), padding='same')
    ])
    return model

model_2 = build_cnn_model_2()
model_2.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 128, 128, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 128, 128, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 128, 128, 3)    │         3,459 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,787 (526.51 KB)

 Trainable params: 134,211 (524.26 KB)

 Non-trainable params: 576 (2.25 KB)

In [13]:
# Compile the model
model_2.compile(optimizer='adam', loss='mean_squared_error')


In [14]:
# Train the model
history_2 = model_2.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 59s 5s/step - loss: 0.7026 - val_loss: 0.1888
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 50s 5s/step - loss: 0.0712 - val_loss: 0.1655
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 51s 5s/step - loss: 0.0238 - val_loss: 0.1388
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 49s 5s/step - loss: 0.0117 - val_loss: 0.1255
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 50s 5s/step - loss: 0.0082 - val_loss: 0.1132
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 50s 5s/step - loss: 0.0065 - val_loss: 0.1001
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 50s 5s/step - loss: 0.0060 - val_loss: 0.0945
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 50s 5s/step - loss: 0.0051 - val_loss: 0.0859
Epoch 9/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 50s 5s/step - loss: 0.0054 - val_loss: 0.0687
Epoch 10/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 49s 5s/step - loss: 0.0063 - val_loss: 0.0686


In [15]:
# Evaluate the model
validation_metrics_2 = evaluate_model(model_2, X_train[:10])  # Using a subset for demonstration
print(validation_metrics_2)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 607ms/step
{'NIQE': 2.9935916472200987, 'PCQI': 2.9566771510148824, 'CEIQ': 3.101611198442365, 'MEME': 2.9366450852713104, 'PSNR': 13.152984476556796, 'SSIM': 0.49423662}


In [16]:
# Update the data with the results of the 2nd model
data['Method'].append('Model 2')
data['NIQE'].append(validation_metrics_2['NIQE'])
data['PCQI'].append(validation_metrics_2['PCQI'])
data['CEIQ'].append(validation_metrics_2['CEIQ'])
data['MEME'].append(validation_metrics_2['MEME'])
data['PSNR'].append(validation_metrics_2['PSNR'])

# Create a new DataFrame with the updated data
df_metrics_updated = pd.DataFrame(data)

# Display the updated DataFrame
print(df_metrics_updated)

# Optional: Apply the same styling to the updated DataFrame
df_styled_updated = df_metrics_updated.style.apply(highlight_best, subset=['NIQE', 'PCQI', 'CEIQ', 'MEME', 'PSNR'])
df_styled_updated


       Method      NIQE      PCQI      CEIQ      MEME       PSNR
0        NBCE  5.291000  1.009000  2.867000  2.888000  21.175000
1        FCCE  4.843000  0.657000  2.620000  6.178000   9.165000
2     SMIRANK  5.100000  0.886000  3.284000  2.663000  22.355000
3         JHE  4.283000  0.935000  3.097000  6.606000  10.565000
4       ESIHE  5.350000  1.009000  3.202000  2.983000  22.946000
5     SSAOWHF  5.121000  0.661000  3.013000  3.219000  19.776000
6        STAR  5.362000  0.969000  3.089000  2.739000  14.943000
7     HAZECSA  5.978000  0.908000  3.007000  2.085000  17.567000
8      FCENII  4.820000  1.011000  3.149000  4.177000  20.750000
9         NPE  4.567000  0.911000  2.897000  3.705000  17.127000
10     OHCICD  5.440000  0.988000  3.163000  2.377000  26.393000
11  FUZZY-DCT  4.361000  1.062000  3.225000  3.375000  23.547000
12    W-CLAHE  5.509000  1.023000  3.325000  4.367000  23.403000
13   Proposed  2.628726  2.919741  3.268920  3.109375  14.231353
14    Model 1  3.045542  

,Method,NIQE,PCQI,CEIQ,MEME,PSNR
0,NBCE,5.291000,1.009000,2.867000,2.888000,21.175000
1,FCCE,4.843000,0.657000,2.620000,6.178000,9.165000
2,SMIRANK,5.100000,0.886000,3.284000,2.663000,22.355000
3,JHE,4.283000,0.935000,3.097000,6.606000,10.565000
4,ESIHE,5.350000,1.009000,3.202000,2.983000,22.946000
5,SSAOWHF,5.121000,0.661000,3.013000,3.219000,19.776000
6,STAR,5.362000,0.969000,3.089000,2.739000,14.943000
7,HAZECSA,5.978000,0.908000,3.007000,2.085000,17.567000
8,FCENII,4.820000,1.011000,3.149000,4.177000,20.750000
9,NPE,4.567000,0.911000,2.897000,3.705000,17.127000


In [17]:
from tensorflow.keras.layers import Add

def residual_block(x, filters):
    skip = x
    x = Conv2D(filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Add()([x, skip])
    x = Activation('relu')(x)
    return x


In [18]:
def build_cnn_model_3(input_shape=(128, 128, 3)):
    inputs = Input(shape=input_shape)
    x = Conv2D(64, (3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = residual_block(x, 64)
    x = residual_block(x, 64)
    x = Conv2D(3, (3, 3), padding='same')(x)
    model = tf.keras.Model(inputs, x)
    return model

model_3 = build_cnn_model_3()
model_3.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 128, 128,  │      1,792 │ input_layer_2[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        256 │ conv2d_8[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_6        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 128, 128,  │     36,928 │ activation_6[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        256 │ conv2d_9[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_7        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_10 (Conv2D)  │ (None, 128, 128,  │     36,928 │ activation_7[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        256 │ conv2d_10[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │ activation_6[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_8        │ (None, 128, 128,  │          0 │ add[0][0]         │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 128, 128,  │     36,928 │ activation_8[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        256 │ conv2d_11[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_9        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_12 (Conv2D)  │ (None, 128, 128,  │     36,928 │ activation_9[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        256 │ conv2d_12[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 128, 128,  │          0 │ batch_normalizat

 Total params: 152,515 (595.76 KB)

 Trainable params: 151,875 (593.26 KB)

 Non-trainable params: 640 (2.50 KB)

In [19]:
# Compile the model
model_3.compile(optimizer='adam', loss='mean_squared_error')


In [20]:
# Train the model
history_3 = model_3.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 68s 6s/step - loss: 2.2459 - val_loss: 0.1793
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 55s 5s/step - loss: 0.1456 - val_loss: 0.1752
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 55s 5s/step - loss: 0.0578 - val_loss: 0.1582
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 55s 6s/step - loss: 0.0362 - val_loss: 0.1495
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 57s 6s/step - loss: 0.0248 - val_loss: 0.1529
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 55s 6s/step - loss: 0.0203 - val_loss: 0.1497
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 55s 5s/step - loss: 0.0178 - val_loss: 0.1515
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 55s 6s/step - loss: 0.0165 - val_loss: 0.1470
Epoch 9/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 56s 6s/step - loss: 0.0151 - val_loss: 0.1482
Epoch 10/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 56s 6s/step - loss: 0.0128 - val_loss: 0.1400


In [21]:
# Evaluate the model
validation_metrics_3 = evaluate_model(model_3, X_train[:10])  # Using a subset for demonstration
print(validation_metrics_3)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 521ms/step
{'NIQE': 3.3639146350333347, 'PCQI': 3.502057482312908, 'CEIQ': 2.9226985344009373, 'MEME': 2.851996087895456, 'PSNR': 9.490771261516471, 'SSIM': 0.25889322}


In [22]:
# Update the data with the results of the 3rd model
data['Method'].append('Model 3')
data['NIQE'].append(validation_metrics_3['NIQE'])
data['PCQI'].append(validation_metrics_3['PCQI'])
data['CEIQ'].append(validation_metrics_3['CEIQ'])
data['MEME'].append(validation_metrics_3['MEME'])
data['PSNR'].append(validation_metrics_3['PSNR'])

# Create a new DataFrame with the updated data
df_metrics_updated = pd.DataFrame(data)

# Display the updated DataFrame
print(df_metrics_updated)

# Optional: Apply the same styling to the updated DataFrame
df_styled_updated = df_metrics_updated.style.apply(highlight_best, subset=['NIQE', 'PCQI', 'CEIQ', 'MEME', 'PSNR'])
df_styled_updated


       Method      NIQE      PCQI      CEIQ      MEME       PSNR
0        NBCE  5.291000  1.009000  2.867000  2.888000  21.175000
1        FCCE  4.843000  0.657000  2.620000  6.178000   9.165000
2     SMIRANK  5.100000  0.886000  3.284000  2.663000  22.355000
3         JHE  4.283000  0.935000  3.097000  6.606000  10.565000
4       ESIHE  5.350000  1.009000  3.202000  2.983000  22.946000
5     SSAOWHF  5.121000  0.661000  3.013000  3.219000  19.776000
6        STAR  5.362000  0.969000  3.089000  2.739000  14.943000
7     HAZECSA  5.978000  0.908000  3.007000  2.085000  17.567000
8      FCENII  4.820000  1.011000  3.149000  4.177000  20.750000
9         NPE  4.567000  0.911000  2.897000  3.705000  17.127000
10     OHCICD  5.440000  0.988000  3.163000  2.377000  26.393000
11  FUZZY-DCT  4.361000  1.062000  3.225000  3.375000  23.547000
12    W-CLAHE  5.509000  1.023000  3.325000  4.367000  23.403000
13   Proposed  2.628726  2.919741  3.268920  3.109375  14.231353
14    Model 1  3.045542  

,Method,NIQE,PCQI,CEIQ,MEME,PSNR
0,NBCE,5.291000,1.009000,2.867000,2.888000,21.175000
1,FCCE,4.843000,0.657000,2.620000,6.178000,9.165000
2,SMIRANK,5.100000,0.886000,3.284000,2.663000,22.355000
3,JHE,4.283000,0.935000,3.097000,6.606000,10.565000
4,ESIHE,5.350000,1.009000,3.202000,2.983000,22.946000
5,SSAOWHF,5.121000,0.661000,3.013000,3.219000,19.776000
6,STAR,5.362000,0.969000,3.089000,2.739000,14.943000
7,HAZECSA,5.978000,0.908000,3.007000,2.085000,17.567000
8,FCENII,4.820000,1.011000,3.149000,4.177000,20.750000
9,NPE,4.567000,0.911000,2.897000,3.705000,17.127000
